In [10]:
import sys
%load_ext autoreload
%autoreload 2
from taskman_client.wrapper3 import report_run3
from trainer_v2.chair_logging import c_log
import tensorflow
from tab_print import tab_print_dict

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import os
os.chdir("/home/youngwookim_umass_edu/code/Chair/")

In [19]:
import sys
import numpy as np
from taskman_client.wrapper3 import report_run3
from trainer_v2.chair_logging import c_log
from trainer_v2.custom_loop.dataset_factories import get_pairwise_dataset_w_score, read_pairwise_as_pointwise
from trainer_v2.custom_loop.definitions import ModelConfig512_1
from trainer_v2.custom_loop.eval_loop import tf_run_eval
from trainer_v2.custom_loop.neural_network_def.ts_concat_probe import TSConcatProbe, ProbeEvaler
from trainer_v2.custom_loop.neural_network_def.two_seg_alt import CombineByScoreAdd
from trainer_v2.custom_loop.per_task.pairwise_trainer import TrainerForLossReturningModel, PairwiseEvaler
from trainer_v2.custom_loop.run_config2 import RunConfig2, get_run_config2
from trainer_v2.custom_loop.train_loop import tf_run_train
from trainer_v2.custom_loop.trainer_if import TrainerIFBase
from trainer_v2.train_util.arg_flags import flags_parser

In [13]:
# -------- Paths --------
model_root = "output/model/runs2"
data_root = "output/tfrecord"

# -------- Names --------
run_name = "mmp_probe1"

config_path = f"data/config_a/{run_name}"

train_file = f"{data_root}/mmp_pep8/*"
eval_file = f"{data_root}/mmp_pep8_val"
output_dir = f"{model_root}/{run_name}/model_12500"

# -------- Command Arguments --------
args = flags_parser.parse_args("")
args.action = "eval"
args.input_files = train_file
args.eval_input_files = eval_file
args.config_path = config_path
args.run_name = run_name
args.init_checkpoint = "output/model/runs/uncased_L-12_H-768_A-12/bert_model.ckpt"
args.output_dir = output_dir
run_config: RunConfig2 = get_run_config2(args)
run_config.print_info()

INFO	chair 	10-16 19:07:51 Overwrite batch_size as 16
WARNING	chair 	10-16 19:07:51 Key 'train_step' is not in the run config
WARNING	chair 	10-16 19:07:51 Key 'save_every_n_step' is not in the run config
WARNING	chair 	10-16 19:07:51 Key 'eval_every_n_step' is not in the run config
WARNING	chair 	10-16 19:07:51 Key 'learning_rate_scheduling' is not in the run config
INFO	chair 	10-16 19:07:51 Overwrite steps_per_execution as 1
INFO	chair 	10-16 19:07:51 CommonRunConfig(batch_size=16, steps_per_execution=1, is_debug_run=False, run_name='mmp_probe1', report_field='', report_condition='', eval_batch_size=None, job_id='')
INFO	chair 	10-16 19:07:51 Model to evaluate: output/model/runs2/mmp_probe1/model_12500
INFO	chair 	10-16 19:07:51 eval_steps: all
INFO	chair 	10-16 19:07:51 {'use_tpu': False, 'tpu_name': None, 'force_use_gpu': False}
INFO	chair 	10-16 19:07:51 {'train_files_path': 'output/tfrecord/mmp_pep8/*', 'eval_files_path': 'output/tfrecord/mmp_pep8_val', 'shuffle_buffer_size': 10

In [14]:
model_config = ModelConfig512_1()
task_model = TSConcatProbe(model_config, CombineByScoreAdd)
run_config.eval_config.eval_step = 10

def build_dataset(input_files, is_for_training):
    return get_pairwise_dataset_w_score(
        input_files, run_config, model_config, is_for_training,
        segment_ids_for_token_type_ids=True)

# if run_config.is_training():
#     trainer: TrainerIFBase = TrainerForLossReturningModel(run_config, task_model)
#     tf_run_train(run_config, trainer, build_dataset)
# else:
#     evaler = ProbeEvaler(run_config)
#     metrics = tf_run_eval(run_config, evaler, build_dataset)
#     tab_print_dict(metrics)

In [15]:
import tensorflow as tf
import os.path
from typing import Dict, Callable

from taskman_client.task_proxy import get_task_manager_proxy
from trainer_v2.chair_logging import c_log
from trainer_v2.custom_loop.evaler_if import EvalerIF
from trainer_v2.custom_loop.modeling_common.tf_helper import distribute_dataset
from trainer_v2.custom_loop.run_config2 import RunConfig2
from trainer_v2.custom_loop.train_loop_helper import fetch_metric_result, get_strategy_from_config
from trainer_v2.custom_loop.neural_network_def.ts_concat_probe import TSConcatProbe

model_config = ModelConfig512_1()

strategy = get_strategy_from_config(run_config)
eval_step = run_config.eval_config.eval_step
steps_per_execution = run_config.common_run_config.steps_per_execution
c_log.debug("Loading model")
task_model = TSConcatProbe(model_config, CombineByScoreAdd)
task_model.build_model(run_config)
encode_model = task_model.define_encode_net()
model_path = run_config.eval_config.model_save_path    
checkpoint = tf.train.Checkpoint(task_model.pair_model)
checkpoint.restore(model_path)

INFO	chair 	10-16 19:07:52 1 GPUs found


In [16]:
from data_generator.tokenizer_wo_tf import get_tokenizer, pretty_tokens
tokenizer = get_tokenizer()

In [17]:
c_log.debug("tf_run_inner initializing dataset")


def rename(record):
    return {
        "input_ids": record["input_ids1"][:, :256],
        "segment_ids": record["segment_ids1"][:, :256],
    }
eval_dataset = build_dataset(run_config.dataset_config.eval_files_path, False)
eval_dataset = eval_dataset.map(rename)
eval_dataset = eval_dataset.take(1000)

In [20]:
from list_lib import flatten 
def decode(record):
    bs = len(record["input_ids"])
    for i in range(bs):
        input_ids = np.array(record["input_ids"][i]).tolist()
        yield tokenizer.convert_ids_to_tokens(input_ids)

    
d = [item for item in iter(eval_dataset)]
d = flatten(map(decode, d))
d = map(lambda t: pretty_tokens(t, True), d)
texts = list(d)
print(len(texts))

16000


In [21]:
for idx, text in enumerate(texts):
    if "when" in text[:30]:
        print(idx, text)

103 [CLS] when did freightliner build [MASK] aluminum cab [SEP] leland [MASK] [MASK] [MASK] freightliner [MASK] built [MASK] [MASK] [MASK] first freightliner [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] trucks [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] 1941 [MASK] [MASK] shovel [MASK] cab [MASK] [MASK] [MASK] 1942 [MASK] [MASK] company [MASK] freightliner [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] 1948 [MASK] [MASK] [MASK] [MASK] trucks [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] freightliner produced [MASK] cab [MASK] [MASK] trucks [MASK] [MASK] aluminum [MASK] [SEP]
184 [CLS] when did gerald [MASK] die [SEP] president ford died on december 26 , 2006 at his home in rancho mirage , california . after ceremonies in california , washington , and grand rapids , he was interred on the grounds of the gerald r . ford presidential museum in grand rapids . [SEP]
231 [CLS] block number when [MASK] cell phone call [SEP] * 67 d

In [22]:
def make_seq_to_dataset():
    seq = ""
    tokens = seq.split()
    ids = []
    for t in tokens:
        if t in tokenizer.vocab:
            ids.append(tokenizer.vocab[t])
        else:
            ids.extend(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(t)))

    seg_ids = []
    cur_seg = 0
    for t in tokens:
        seg_ids.append(cur_seg)
        if t == 102:
            cur_seg = 1 - cur_seg
    seq_len = 256
    num_pad = seq_len - len(ids)
    ids = ids + [0] * num_pad
    seg_ids = seg_ids + [0] * num_pad

    def generator():
        yield (ids, seg_ids),
    SpecI = tf.TensorSpec([max_seq_length], dtype=tf.int32)
    sig = (SpecI, SpecI,),
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=sig)
    return dataset




In [ ]:
n_ds = make_seq_to_dataset()

n_ds = make_seq_to_dataset()

concat_ds = n_ds.concatenate(eval_dataset)

In [23]:
def reform(ret):
    n_layer = 13
    n_item = len(ret["rep_out_list"][0])
    keys = ["rep_out_list", "mask_out_list", "probe_out_list"]
    output = []
    for item_idx in range(n_item):
        d = {}
        for key in keys:
            for layer_idx in range(n_layer):
                d[f"{key}_{layer_idx}"] = ret[key][layer_idx][item_idx]
        output.append(d)
    return output

In [24]:
ret = encode_model.predict(eval_dataset)
ret_reform_d = reform(ret)

In [25]:
src_i = 523 
print(texts[src_i])

[CLS] what does it mean when we [MASK] is has an antiparallel structure ? [SEP] what is meant by antiparallel strands as it refers to the structure of dna ? dna ' s structure is a double helix , so there are two sides . one side runs [MASK] [MASK] direction [MASK] [MASK] [MASK] [MASK] [MASK] and the other [MASK] runs [MASK] the opposite direction [MASK] [MASK] [MASK] [MASK] [MASK] . [SEP]


In [28]:
import faiss
import numpy as np
def exp(key):
    reps = np.array([e[key] for e in ret_reform_d])
    d = len(reps[0])
    print("num_items", len(reps))
    index = faiss.IndexFlatL2(d)   # build the index
    index.add(reps)                  # add vectors to the index
    scores, indices = index.search(reps[src_i:src_i+1], k=10)
    for s, i in zip(scores[0], indices[0]):
        print(texts[i], s)

In [29]:
exp("mask_out_list_11")

num_items 16000
[CLS] what does it mean when we [MASK] is has an antiparallel structure ? [SEP] what is meant by antiparallel strands as it refers to the structure of dna ? dna ' s structure is a double helix , so there are two sides . one side runs [MASK] [MASK] direction [MASK] [MASK] [MASK] [MASK] [MASK] and the other [MASK] runs [MASK] the opposite direction [MASK] [MASK] [MASK] [MASK] [MASK] . [SEP] 0.0
[CLS] which condition is hardening of arteries due to a buildup of cholesterol [MASK] ? [SEP] hardening of the arteries due to cholesterol plaques is called atherosclerosis or arteriosclerosis . [MASK] term atherosclerosis literally means hardening from atheroma . [MASK] a he [MASK] ##¦ [MASK] ##thy person [MASK] [MASK] artiers have walls [MASK] [MASK] muscular [MASK] smooth [MASK] elastic [MASK] [MASK] [MASK] [MASK] variations [MASK] blood pressure [MASK] which allows [MASK] blood [MASK] pass through freely . [SEP] 0.008720395
[CLS] what is amorphous [MASK] [SEP] amorphous urates 

In [30]:
exp("rep_out_list_11")

num_items 16000
[CLS] what does it mean when we [MASK] is has an antiparallel structure ? [SEP] what is meant by antiparallel strands as it refers to the structure of dna ? dna ' s structure is a double helix , so there are two sides . one side runs [MASK] [MASK] direction [MASK] [MASK] [MASK] [MASK] [MASK] and the other [MASK] runs [MASK] the opposite direction [MASK] [MASK] [MASK] [MASK] [MASK] . [SEP] 0.0
[CLS] which condition is hardening of arteries due to a buildup of cholesterol [MASK] ? [SEP] hardening of the arteries due to cholesterol plaques is called atherosclerosis or arteriosclerosis . [MASK] term atherosclerosis literally means hardening from atheroma . [MASK] a he [MASK] ##¦ [MASK] ##thy person [MASK] [MASK] artiers have walls [MASK] [MASK] muscular [MASK] smooth [MASK] elastic [MASK] [MASK] [MASK] [MASK] variations [MASK] blood pressure [MASK] which allows [MASK] blood [MASK] pass through freely . [SEP] 2.9321704
[CLS] what is the function of helicase [MASK] replicatio

In [32]:
exp("rep_out_list_5")

num_items 16000
[CLS] what does it mean when we [MASK] is has an antiparallel structure ? [SEP] what is meant by antiparallel strands as it refers to the structure of dna ? dna ' s structure is a double helix , so there are two sides . one side runs [MASK] [MASK] direction [MASK] [MASK] [MASK] [MASK] [MASK] and the other [MASK] runs [MASK] the opposite direction [MASK] [MASK] [MASK] [MASK] [MASK] . [SEP] 0.0
[CLS] what is [MASK] word sanction [SEP] what is sanction ? in the original sense of the word , a asanctiona is a penalty or punishment provided as a means of enforcing obedience to a law . in jurisprudence , a law is said to have a sanction when there is a state which will intervene if it is disobeyed or disregarded . therefore international law has no legal sanction . [SEP] 25.606642
[CLS] what [MASK] are epithelial tissues classified [SEP] how epithelial tissues are classified : epithelia are classified on the basis of cell shape and arrangement . the cell shapes are columnar ( 

In [31]:
exp("mask_out_list_5")

num_items 16000
[CLS] what does it mean when we [MASK] is has an antiparallel structure ? [SEP] what is meant by antiparallel strands as it refers to the structure of dna ? dna ' s structure is a double helix , so there are two sides . one side runs [MASK] [MASK] direction [MASK] [MASK] [MASK] [MASK] [MASK] and the other [MASK] runs [MASK] the opposite direction [MASK] [MASK] [MASK] [MASK] [MASK] . [SEP] 0.0
[CLS] what is [MASK] word sanction [SEP] what is sanction ? in the original sense of the word , a asanctiona is a penalty or punishment provided as a means of enforcing obedience to a law . in jurisprudence , a law is said to have a sanction when there is a state which will intervene if it is disobeyed or disregarded . therefore international law has no legal sanction . [SEP] 0.061350346
[CLS] what [MASK] are epithelial tissues classified [SEP] how epithelial tissues are classified : epithelia are classified on the basis of cell shape and arrangement . the cell shapes are columnar 

In [147]:
from data_generator.tokenizer_wo_tf import get_tokenizer
from data_generator2.segmented_enc.hf_encode_helper import combine_with_sep_cls_and_pad
max_seq_length = 256
batch_size = 4
tokenizer = get_tokenizer()
def score_fn(qd_list: list[tuple[str, str]]) -> list[float]:
    def generator():
        for q_term, d_term in qd_list:
            q_term_tokens = tokenizer.tokenize(q_term)
            d_term_tokens = tokenizer.tokenize(d_term)
            input_ids, segment_ids = combine_with_sep_cls_and_pad(
                tokenizer, q_term_tokens, d_term_tokens, max_seq_length)
            yield (input_ids, segment_ids),

    SpecI = tf.TensorSpec([max_seq_length], dtype=tf.int32)
    sig = (SpecI, SpecI,),
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=sig)
    dataset = dataset.batch(batch_size)
    return encode_model.predict(dataset)
    

In [148]:
pair_list = [('car', 'hyundai'),
 ('car', 'volkswagen'),
 ('car', 'hyundai'),
 ('car', 'volkswagen')]


In [149]:
def generator():
    for q_term, d_term in pair_list:
        q_term_tokens = tokenizer.tokenize(q_term)
        d_term_tokens = tokenizer.tokenize(d_term)
        input_ids, segment_ids = combine_with_sep_cls_and_pad(
            tokenizer, q_term_tokens, d_term_tokens, max_seq_length)
        yield (input_ids, segment_ids),
with strategy.scope():
    SpecI = tf.TensorSpec([max_seq_length], dtype=tf.int32)
    sig = (SpecI, SpecI,),
    dataset = tf.data.Dataset.from_generator(
        generator,
        output_signature=sig)
    b_dataset = dataset.batch(batch_size)
    itr = iter(b_dataset)
    batch = next(itr)
    encode_model(batch)

In [151]:
ret = score_fn(pair_list)

In [189]:
len(ret["rep_out_list"])

13

In [11]:
import faiss
import numpy as np
import os
os.chdir("/home/youngwookim_umass_edu/code/Chair/")

In [4]:
import pickle
file_path = "output/mmp/car_mmp_probe1_12500/0.pkl"

obj = pickle.load(open(file_path, "rb"))


In [5]:
keys = [r[1] for r in obj]

In [6]:
print(keys)

['vehiclea', 'vehiclesa', 'vehicleas', 'carman', 'carf', 'carsa', 'carmaker', 'sedan', 'carmakers', 'sedans', 'fiat', 'autos', 'cadillacs', 'saloons', 'motorcar', 'carinii', '805', '316', '340b', '316l', '725', '815', '705', '795', 'ferrari', '318', 'bicarb', 'camaros', 'chevrolets', 'ascarids', 'ferraris', '715', '695', '535', 'toyotas', '935', '515', 'carville', 'volvo', '605', '755', 'renumbered', '855', '905', '635', '39s', '400s', '775', '685', 'carnallite', '808', '835', '698', '718', 'limousines', 'carotids', 'caro', '798', '895', '735', '758', '730', '807', '425', '340', '785', 'triumph', '545', 'dion', '945', '765', '325i', '289', '838', 'caritas', '724', 'numbered', 'compacts', '704', '818', '975', '804', '727', '955', '728', '665', '638', '615', 'overturned', 'cadillac', '995', '925', '335', '1040ez', '778', '708', 'engined', '351', 'fords', '406', '794', '757', 'motorised', '797', '3125', '475', '678', '675', '898', '518', '858', '575', 'mazda3', '801', '435', 'supercar', '

In [17]:
word = "mustangs"
src_i = keys.index(word)

In [18]:
def exp(key):
    reps = np.array([r[2][key] for r in obj])
    d = len(reps[0])
    index = faiss.IndexFlatL2(d)   # build the index
    index.add(reps)                  # add vectors to the index
    scores, indices = index.search(reps[src_i:src_i+1], k=10)
    for s, i in zip(scores[0], indices[0]):
        print(keys[i], s)

In [19]:
exp("mask_out_list_11")

mustangs 0.0
interceptors 0.0053369086
racetracks 0.0060172356
dodges 0.0062560383
forerunners 0.0062892176
gts 0.0064644567
raceways 0.006481085
v8s 0.006527315
lineups 0.0065407166
trolleys 0.0066814465


In [20]:
exp("rep_out_list_11")

mustangs 0.0
mustang 0.7172456
toros 1.0624194
pintos 1.1546907
v8s 1.1796956
triumphs 1.1818429
lincolns 1.1873202
fordas 1.2244132
roadrunner 1.3252296
lamborghinis 1.3537021


In [21]:
exp("mask_out_list_4")

mustangs 0.0
mustang 0.0036902896
fords 0.00605145
dodges 0.0065913163
chevrolets 0.007068475
coupes 0.007315962
toyotas 0.0073179095
chevy 0.008458303
minivans 0.008573349
toros 0.0087733315


In [22]:
exp("mask_out_list_11")

mustangs 0.0
mustang 0.0020111096
toros 0.002712542
triumphs 0.0028118948
lincolns 0.0028395022
pintos 0.0028696316
v8s 0.0029338726
fordas 0.0029751533
a90s 0.0033682503
lamborghinis 0.0034191504


In [205]:
key = "rep_out_list_12"
print(d_terms)
for term1 in d_terms:
    row = []
    for term2 in d_terms:
        score = np.dot(reps[term1], reps[term2])
        row.append(score)
    print(row)


['vehiclea', 'vehiclesa', 'vehicleas', 'carman', 'carf', 'carsa', 'carmaker', 'sedan', 'carmakers', 'sedans', 'fiat', 'autos', 'cadillacs', 'saloons', 'motorcar', 'carinii', '805', '316', '340b', '316l', '725', '815', '705', '795', 'ferrari', '318', 'bicarb', 'camaros', 'chevrolets', 'ascarids', 'ferraris', '715', '695', '535', 'toyotas', '935', '515', 'carville', 'volvo', '605', '755', 'renumbered', '855', '905', '635', '39s', '400s', '775', '685', 'carnallite', '808', '835', '698', '718', 'limousines', 'carotids', 'caro', '798', '895', '735', '758', '730', '807', '425', '340', '785', 'triumph', '545', 'dion', '945', '765', '325i', '289', '838', 'caritas', '724', 'numbered', 'compacts', '704', '818', '975', '804', '727', '955', '728', '665', '638', '615', 'overturned', 'cadillac', '995', '925', '335', '1040ez', '778', '708', 'engined', '351', 'fords', '406']
[206.93187, 208.14444, 206.7114, 203.37056, 207.0513, 208.26907, 204.34915, 206.9176, 203.63147, 206.20117, 205.78316, 204.92798

In [188]:
obj[1][2]["rep_out_list"].shape

(10, 768)

In [195]:
obj[0][2].keys()

dict_keys(['rep_out_list_0', 'rep_out_list_1', 'rep_out_list_2', 'rep_out_list_3', 'rep_out_list_4', 'rep_out_list_5', 'rep_out_list_6', 'rep_out_list_7', 'rep_out_list_8', 'rep_out_list_9', 'rep_out_list_10', 'rep_out_list_11', 'mask_out_list_0', 'mask_out_list_1', 'mask_out_list_2', 'mask_out_list_3', 'mask_out_list_4', 'mask_out_list_5', 'mask_out_list_6', 'mask_out_list_7', 'mask_out_list_8', 'mask_out_list_9', 'mask_out_list_10', 'mask_out_list_11', 'probe_out_list_0', 'probe_out_list_1', 'probe_out_list_2', 'probe_out_list_3', 'probe_out_list_4', 'probe_out_list_5', 'probe_out_list_6', 'probe_out_list_7', 'probe_out_list_8', 'probe_out_list_9', 'probe_out_list_10', 'probe_out_list_11'])

In [199]:
np.dot

<function numpy.dot>